# Task 2: Fine-Tuning T5-base untuk Generative Question Answering

Notebook ini menjelaskan proses fine-tuning model Transformer encoder–decoder **T5-base** untuk menyelesaikan tugas *generative question answering* menggunakan dataset **SQuAD**.

**Tujuan:**

* Memuat dan melakukan pra-pemrosesan dataset SQuAD
* Menyusun format input menjadi *"question: <pertanyaan> context: <konteks>"*
* Melakukan fine-tuning model T5-base menggunakan Hugging Face Trainer
* Mengevaluasi performa model menggunakan metrik Exact Match (EM) dan F1-score
* Menghasilkan jawaban untuk beberapa contoh pertanyaan

**Requirements:**

* transformers, datasets, evaluate, torch, pandas



# Import Requirements Libraries

Mengimport library yang dibutuhkan untuk training model, data processing, dan evaluasi metrik.

In [3]:
!pip install transformers[torch] datasets evaluate

import warnings
warnings.filterwarnings('ignore')

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


## Model Configuration and Dataset Preparation

Pada tahap ini, dilakukan konfigurasi awal yang mencakup penentuan model, parameter panjang input dan output, serta pemuatan dataset yang akan digunakan dalam proses fine-tuning.

Model yang digunakan adalah **T5-base**, yaitu model Transformer berarsitektur encoder–decoder yang telah dilatih sebelumnya dan cocok untuk tugas *sequence-to-sequence* seperti *generative question answering*. Panjang maksimum input ditetapkan sebesar 512 token untuk menampung gabungan konteks dan pertanyaan, sedangkan panjang maksimum output ditetapkan sebesar 32 token karena jawaban pada dataset SQuAD umumnya bersifat singkat.

Dataset yang digunakan adalah **SQuAD (Stanford Question Answering Dataset)**, yang terdiri dari beberapa kolom utama seperti *context*, *question*, dan *answers*. Dataset ini dimuat menggunakan pustaka Hugging Face Datasets. Untuk efisiensi proses pelatihan dan evaluasi, hanya sebagian data yang digunakan, yaitu 1000 data untuk pelatihan dan 100 data untuk validasi.

Selain itu, tokenizer yang sesuai dengan model T5-base juga dimuat untuk mengonversi teks input dan output menjadi representasi token numerik. Dataset kemudian dikonversi ke dalam format *Pandas DataFrame* untuk keperluan analisis awal, termasuk pemeriksaan jumlah data, struktur kolom, dan contoh isi dataset.


In [4]:
# 1. Konfigurasi
MODEL_NAME = "t5-base"  # Model pre-trained
MAX_INPUT_LENGTH = 512        # Panjang maksimal input (konteks + pertanyaan)
MAX_TARGET_LENGTH = 32        # Panjang maksimal output (jawaban)

# 2. Load Dataset SQuAD
# SQuAD berisi kolom: 'id', 'title', 'context', 'question', 'answers'
raw_datasets = load_dataset("squad")
print("Contoh data:", raw_datasets["train"][0])
train_subset = raw_datasets["train"].select(range(1000))
val_subset = raw_datasets["validation"].select(range(100))

# 3. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("Berhasil import dan load dataset!")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Contoh data: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Berhasil import dan load dataset!


In [5]:
# Konversi dataset 'train' ke Pandas DataFrame untuk analisis
df_train = train_subset.to_pandas()
df_val   = val_subset.to_pandas()

print("=== INFO DATASET ===")
print(f"Jumlah Data Train: {df_train.shape[0]} baris")
print(f"Jumlah Data Val  : {df_val.shape[0]} baris")
print("\n=== SAMPLE DATA ===")
display(df_train.head(3)) # Menampilkan 3 baris pertama
print("\n=== KOLOM & TIPE DATA ===")
print(df_train.info())

=== INFO DATASET ===
Jumlah Data Train: 1000 baris
Jumlah Data Val  : 100 baris

=== SAMPLE DATA ===


,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."



=== KOLOM & TIPE DATA ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1000 non-null   object
 1   title     1000 non-null   object
 2   context   1000 non-null   object
 3   question  1000 non-null   object
 4   answers   1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB
None


In [6]:
df_train = df_train[["question", "context", "answers"]]
df_train.head()

,question,context,answers
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...","{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...","{'text': ['a copper statue of Christ'], 'answe..."
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...","{'text': ['the Main Building'], 'answer_start'..."
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",{'text': ['a Marian place of prayer and reflec...
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",{'text': ['a golden statue of the Virgin Mary'...


In [7]:
df_train["answer_text"] = df_train["answers"].apply(lambda x: x["text"][0])
df_train = df_train.drop(columns=["answers"])
df_train.head()

,question,context,answer_text
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",the Main Building
3,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",a golden statue of the Virgin Mary


In [8]:
df_train.describe()

,question,context,answer_text
count,1000,1000,1000
unique,998,119,712
top,When did Beyonce have her first child?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Jay Z
freq,2,20,11


## Pre-Processing Data and Tokenization

Tahap ini bertujuan untuk menyiapkan data agar sesuai dengan format input dan output yang dibutuhkan oleh model T5-base pada tugas *generative question answering*.

**Langkah-langkah pra-pemrosesan yang dilakukan:**

* Menggabungkan pertanyaan dan konteks ke dalam satu input teks dengan format
  *"question: <pertanyaan> context: <konteks>"* sesuai dengan konvensi model T5.
* Menggunakan jawaban pertama pada dataset SQuAD sebagai target teks untuk proses pelatihan.
* Melakukan tokenisasi pada input dan target dengan panjang maksimum yang telah ditentukan.
* Menerapkan *truncation* untuk membatasi panjang urutan dan *padding* agar seluruh data memiliki panjang yang seragam.
* Mengganti token *padding* pada label menjadi **-100** agar tidak diperhitungkan dalam perhitungan loss selama training.

Fungsi pra-pemrosesan ini kemudian diterapkan ke seluruh data pelatihan dan validasi menggunakan metode *map*. Dataset hasil tokenisasi inilah yang selanjutnya digunakan pada tahap pelatihan dan evaluasi model.


In [9]:
def preprocess_function(examples):
    inputs = []
    targets = []

    # Loop melalui setiap data
    for i in range(len(examples["context"])):
        # Format input khusus T5
        input_text = f"question: {examples['question'][i]} context: {examples['context'][i]}"
        inputs.append(input_text)

        # Ambil teks jawaban pertama (SQuAD bisa punya beberapa referensi jawaban, ambil yang pertama untuk training)
        targets.append(examples["answers"][i]["text"][0])

    # Tokenisasi Input
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, padding="max_length")

    # Tokenisasi Target (Jawaban)
    labels = tokenizer(targets, max_length=MAX_TARGET_LENGTH, truncation=True, padding="max_length")

    # Ganti padding token di label menjadi -100 agar tidak dihitung dalam loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Terapkan preprocessing ke seluruh dataset (Train & Validation)

tokenized_train = train_subset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_subset.column_names
)

tokenized_val = val_subset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_subset.column_names
)

print(f"Training samples: {len(tokenized_train)}")
print(f"Validation samples: {len(tokenized_val)}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Training samples: 1000
Validation samples: 100


## Training Model (Fine-Tuning)

Pada tahap ini, model **T5-base** dilakukan proses fine-tuning menggunakan subset dataset SQuAD yang telah dipra-pemrosesan sebelumnya. Proses pelatihan dilakukan dengan memanfaatkan kerangka kerja **Seq2SeqTrainer** dari Hugging Face untuk menangani tugas *generative question answering*.

**Konfigurasi utama pelatihan:**

* Model dilatih selama **2 epoch** untuk menjaga keseimbangan antara performa dan efisiensi komputasi.
* Ukuran *batch* ditetapkan sebesar **8** untuk data pelatihan dan validasi.
* Opsi `predict_with_generate` dinonaktifkan selama training untuk mempercepat proses dan mengurangi penggunaan memori GPU.
* *Mixed precision training* (`fp16`) digunakan apabila GPU tersedia untuk meningkatkan efisiensi komputasi.

Dataset pelatihan dan validasi yang digunakan merupakan **subset data**, sehingga proses training dapat berjalan lebih cepat tanpa menghilangkan tujuan utama eksperimen. Setelah proses pelatihan selesai, metrik dasar seperti *training loss*, waktu pelatihan, dan kecepatan pemrosesan data ditampilkan sebagai indikator performa training.


In [10]:
# 1. Load Model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# 2. Training Arguments
args = Seq2SeqTrainingArguments(
    output_dir="./t5-squad-subset-2000",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=2,
    predict_with_generate=False,
    fp16=torch.cuda.is_available(),
    logging_steps=20,
    report_to="none"
)

# 3. Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 4. Inisialisasi Trainer (SUDAH DISESUAIKAN VARIABELNYA)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train, # Changed from tokenized_datasets["train"]
    eval_dataset=tokenized_val,    # Changed from tokenized_datasets["validation"]
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Trainer siap dengan dataset subset!")

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Trainer siap dengan dataset subset!


In [11]:
# Train the model
print("Starting training...")
train_result = trainer.train()
print("Training completed!")

# Print training metrics
print("\nTraining Metrics:")
print(f"Training Loss: {train_result.training_loss:.4f}")
print(f"Training Runtime: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Training Samples/Second: {train_result.metrics['train_samples_per_second']:.2f}")

Starting training...


Epoch,Training Loss,Validation Loss
1,0.272100,0.372506
2,0.216300,0.359023


Training completed!

Training Metrics:
Training Loss: 0.2578
Training Runtime: 169.16 seconds
Training Samples/Second: 11.82


In [12]:
from google.colab import drive
drive.mount("/content/drive")
OUTPUT_DIR = "/content/drive/MyDrive/t5-question-answer"
model.save_pretrained(f"{OUTPUT_DIR}/model_trained")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/model_trained")

print(f"Model saved to Google Drive at: {OUTPUT_DIR}/model_trained")

Mounted at /content/drive
Model saved to Google Drive at: /content/drive/MyDrive/t5-question-answer/model_trained


## Metrics Evaluation Question Answering (Exact Match & F1)

Pada tahap ini, didefinisikan fungsi evaluasi untuk mengukur performa model pada tugas *generative question answering* menggunakan metrik **Exact Match (EM)** dan **F1 Score**, yang merupakan metrik standar pada dataset SQuAD.

Metrik **Exact Match** mengukur persentase prediksi yang sama persis dengan jawaban referensi, sedangkan **F1 Score** mengukur tingkat kesesuaian antara prediksi dan jawaban referensi berdasarkan tumpang tindih token. Karena evaluasi SQuAD memerlukan format input tertentu, hasil prediksi dan label referensi terlebih dahulu disesuaikan dengan struktur yang dibutuhkan oleh modul evaluasi.

Fungsi evaluasi ini dibuat secara terpisah agar dapat digunakan kembali pada tahap evaluasi model setelah proses inferensi, sehingga alur eksperimen menjadi lebih terstruktur dan modular.


In [13]:
# Load evaluation metric for SQuAD
qa_metric = evaluate.load("squad")

def compute_qa_metrics(eval_output):
    """
    Menghitung Exact Match dan F1 Score
    untuk task Question Answering berbasis SQuAD.
    """
    preds, gold_labels = eval_output

    # Decode hasil prediksi model
    decoded_predictions = tokenizer.batch_decode(
        preds, skip_special_tokens=True
    )

    # Ganti label -100 dengan padding token agar bisa di-decode
    gold_labels = np.where(
        gold_labels != -100,
        gold_labels,
        tokenizer.pad_token_id
    )
    decoded_references = tokenizer.batch_decode(
        gold_labels, skip_special_tokens=True
    )

    # Format output sesuai kebutuhan metric SQuAD
    prediction_list = [
        {"id": str(idx), "prediction_text": pred.strip()}
        for idx, pred in enumerate(decoded_predictions)
    ]

    reference_list = [
        {
            "id": str(idx),
            "answers": {
                "text": [ref.strip()],
                "answer_start": [0]
            }
        }
        for idx, ref in enumerate(decoded_references)
    ]

    # Hitung Exact Match dan F1
    result = qa_metric.compute(
        predictions=prediction_list,
        references=reference_list
    )

    return {
        "exact_match": result["exact_match"],
        "f1": result["f1"]
    }

print("QA metric function siap digunakan!")


QA metric function siap digunakan!


## Generasi Jawaban dan Evaluasi Model

Pada tahap ini, dilakukan proses inferensi pada data validasi dengan menghasilkan jawaban secara langsung menggunakan model yang telah dilatih. Model dijalankan dalam mode evaluasi dan perhitungan gradien dinonaktifkan untuk meningkatkan efisiensi komputasi serta menghemat penggunaan memori.

Data validasi diproses secara bertahap menggunakan *DataLoader* untuk mengatur proses batching dan padding secara otomatis. Model kemudian menghasilkan jawaban untuk setiap pasangan pertanyaan dan konteks menggunakan metode *text generation*. Seluruh hasil prediksi yang dihasilkan selanjutnya dievaluasi menggunakan fungsi metrik yang telah didefinisikan sebelumnya.

### Hasil Evaluasi Model

| Metrik      | Nilai |
| ----------- | ----- |
| Exact Match | 80.00 |
| F1 Score    | 83.05 |

Berdasarkan hasil evaluasi tersebut, model menunjukkan kemampuan yang baik dalam menghasilkan jawaban yang relevan dan akurat terhadap pertanyaan yang diberikan, baik dari segi kesamaan teks secara persis maupun kesesuaian makna jawaban.


In [15]:
# Tentukan device yang digunakan
run_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(run_device)

print("Generating predictions on validation set in batches...")

# Data collator untuk Seq2Seq
eval_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# DataLoader untuk validation set
eval_loader = torch.utils.data.DataLoader(
    tokenized_val,
    batch_size=args.per_device_eval_batch_size,
    shuffle=False,
    collate_fn=eval_collator
)

# Penampung hasil prediksi dan label
generated_sequences = []
reference_labels = []

# Mode evaluasi
model.eval()

with torch.no_grad():
    for eval_batch in tqdm(eval_loader, desc="Predicting"):
        # Pindahkan input ke device
        batch_input_ids = eval_batch["input_ids"].to(run_device)
        batch_attention = eval_batch["attention_mask"].to(run_device)

        # Generate jawaban
        output_ids = model.generate(
            input_ids=batch_input_ids,
            attention_mask=batch_attention,
            max_length=MAX_TARGET_LENGTH,
            num_beams=4,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id
        )

        # Simpan hasil generate (variable length)
        generated_sequences.extend(output_ids.cpu().tolist())

        # Simpan label referensi
        reference_labels.append(eval_batch["labels"].cpu().numpy())

# Padding hasil prediksi agar panjangnya seragam
padded_outputs = tokenizer.pad(
    {"input_ids": generated_sequences},
    padding="longest",
    return_tensors="np"
)
pred_ids = padded_outputs["input_ids"]

# Gabungkan seluruh label menjadi satu array
label_ids = np.concatenate(reference_labels, axis=0)

# Hitung metrik evaluasi
eval_metrics = compute_qa_metrics((pred_ids, label_ids)) # Corrected function name

print("\n=== Evaluation Results ===")
print(f"Exact Match: {eval_metrics['exact_match']:.2f}")
print(f"F1 Score: {eval_metrics['f1']:.2f}")

Generating predictions on validation set in batches...


Predicting:   0%|          | 0/13 [00:00<?, ?it/s]


=== Evaluation Results ===
Exact Match: 80.00
F1 Score: 83.05


## Qualitative Evaluation on Validation Data
Bagian ini menampilkan contoh hasil inferensi model pada beberapa data validasi yang dipilih secara manual. Tujuannya adalah untuk memberikan gambaran kualitatif mengenai kemampuan model dalam menghasilkan jawaban berdasarkan pasangan pertanyaan dan konteks yang diberikan.

Model menerima input berupa gabungan teks pertanyaan dan konteks, kemudian menghasilkan jawaban secara generatif menggunakan mekanisme beam search. Beberapa indeks data validasi dipilih untuk memastikan bahwa contoh yang ditampilkan berasal dari konteks yang berbeda.

Hasil prediksi yang dihasilkan dibandingkan secara langsung dengan jawaban referensi (ground truth) untuk menunjukkan tingkat akurasi model secara kualitatif

In [19]:
def generate_testing(q_text, c_text):
    """
    Menghasilkan jawaban dari pertanyaan dan konteks.
    """
    prompt_text = f"question: {q_text} context: {c_text}"

    encoded_input = tokenizer(
        prompt_text,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(run_device) # Changed to run_device

    generated_output = model.generate(
        **encoded_input,
        max_length=64,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(
        generated_output[0],
        skip_special_tokens=True
    )


# Contoh inferensi pada beberapa data validasi
indices = [22, 41, 65]

for idx in indices:
    sample_data = val_subset[idx]

    predicted = generate_testing(
        sample_data["question"],
        sample_data["context"]
    )

    print(f"\nExample {idx}")
    print(f"Question        : {sample_data['question']}")
    print(f"Context Preview : {sample_data['context'][:200]}...")
    print(f"Ground Truth    : {sample_data['answers']['text'][0]}")
    print(f"Prediction      : {predicted}")


Example 22
Question        : What team was the AFC champion?
Context Preview : Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated...
Ground Truth    : Denver Broncos
Prediction      : Denver Broncos

Example 41
Question        : Which Carolina Panthers team member was picked as the team's MVP in 2015? 
Context Preview : The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship ...
Ground Truth    : Cam Newton
Prediction      : Cam Newton

Example 65
Question        : Who was given the esteemed status of MVP for Super Bowl 50?
Context Preview : The Broncos took an early lead in Super Bowl 50 and never trailed. Newton was limited by Denver's defense, which sacked him seven times and forced him into t

## Manual Question Answering Example (Custom Input)

Bagian ini menampilkan contoh penggunaan model *generative question answering* dengan input yang diberikan secara manual, tanpa menggunakan data dari dataset SQuAD. Tujuan dari contoh ini adalah untuk menunjukkan bahwa model tidak hanya bekerja pada data validasi, tetapi juga mampu menjawab pertanyaan baru berdasarkan konteks yang diberikan oleh pengguna.

Model menerima masukan berupa teks konteks dan pertanyaan dalam bahasa alami, kemudian menghasilkan jawaban secara generatif menggunakan mekanisme *sequence-to-sequence*. Contoh ini memberikan ilustrasi tambahan mengenai fleksibilitas model dalam menjawab pertanyaan di luar data pelatihan.

Hasil yang diperoleh menunjukkan bahwa model mampu mengekstrak informasi penting dari konteks dan menghasilkan jawaban yang relevan dan sesuai dengan pertanyaan yang diajukan.


In [27]:
# Fungsi untuk prediksi manual
def ask_question(question, context):
    # 1. Format input
    input_text = f"question: {question} context: {context}"

    # 2. Tokenisasi
    inputs = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda") # Pastikan ke GPU jika pakai GPU

    # 3. Generate Jawaban
    outputs = model.generate(inputs, max_length=32)

    # 4. Decode hasil token menjadi teks
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# --- CONTOH PENGGUNAAN ---
my_context = """
Monas atau Monumen Nasional adalah ikon kota Jakarta yang terletak di pusat kota.
Tugu ini dibangun untuk mengenang perlawanan dan perjuangan rakyat Indonesia untuk merebut kemerdekaan
dari pemerintahan kolonial Hindia Belanda. Pembangunan dimulai pada tanggal 17 Agustus 1961.
"""

my_question = "Kapan pembangunan Monas dimulai?"

print("Konteks:", my_context)
print("Pertanyaan:", my_question)
print("-" * 30)
print("Jawaban Model:", ask_question(my_question, my_context))

Konteks: 
Monas atau Monumen Nasional adalah ikon kota Jakarta yang terletak di pusat kota.
Tugu ini dibangun untuk mengenang perlawanan dan perjuangan rakyat Indonesia untuk merebut kemerdekaan
dari pemerintahan kolonial Hindia Belanda. Pembangunan dimulai pada tanggal 17 Agustus 1961.

Pertanyaan: Kapan pembangunan Monas dimulai?
------------------------------
Jawaban Model: pada tanggal 17 Agustus 1961.
